In [1]:
# # 02 — Косинусное сходство (query ↔ title)
# - Источник эмбеддингов: np.memmap float16 (из 01_make_embeddings.ipynb)
# - Результат:
#     - train_cos.parquet:  [query_id, item_id, cos_q_title]
#     - test_cos.parquet:   [query_id, item_id, cos_q_title]
# - Прогресс: tqdm

In [2]:
import os
import json
from typing import List, Tuple

import polars as pl
import numpy as np
from tqdm.auto import tqdm

C:\Users\idine\PycharmProjects\Avito_Test\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, "Data")
TRAIN_PATH = os.path.join(DATA_DIR, "train-dset.parquet")
TEST_PATH  = os.path.join(DATA_DIR, "test-dset-small.parquet")

In [4]:
EMB_DIR    = os.path.join(DATA_DIR, "embeddings_memmap")
QUERY_MM   = os.path.join(EMB_DIR, "query_embeddings.f16.memmap")
ITEM_MM    = os.path.join(EMB_DIR, "item_embeddings.f16.memmap")
Q_INDEX    = os.path.join(EMB_DIR, "query_id2idx.json")
I_INDEX    = os.path.join(EMB_DIR, "item_id2idx.json")

In [5]:
EMB_DIM    = 384

In [6]:
CHUNK_ROWS = 2_000_000

In [7]:
# Выходные файлы
TRAIN_COS_OUT = "train_cos.parquet"
TEST_COS_OUT  = "test_cos.parquet"

In [8]:
print(f"[setup] chunk_rows={CHUNK_ROWS}, emb_dim={EMB_DIM}")
print(f"[setup] embeddings: {QUERY_MM}, {ITEM_MM}")

[setup] chunk_rows=2000000, emb_dim=384
[setup] embeddings: C:\Users\idine\PycharmProjects\Avito_Test\Data\embeddings_memmap\query_embeddings.f16.memmap, C:\Users\idine\PycharmProjects\Avito_Test\Data\embeddings_memmap\item_embeddings.f16.memmap


In [9]:
# ## 1) Загрузка индексов и memmap

# Загружаем словари id → индекс в меммапе
with open(Q_INDEX, "r", encoding="utf-8") as f:
    q_id2idx = json.load(f)  # ключи JSON — строки
with open(I_INDEX, "r", encoding="utf-8") as f:
    i_id2idx = json.load(f)

In [10]:
# Создаём memmap массивы. Размер по первой оси = число ID в словаре.
q_count = len(q_id2idx)
i_count = len(i_id2idx)

In [11]:
q_mm = np.memmap(QUERY_MM, dtype="float16", mode="r", shape=(q_count, EMB_DIM))
i_mm = np.memmap(ITEM_MM,  dtype="float16", mode="r", shape=(i_count, EMB_DIM))

print(f"[memmap] queries: shape=({q_count},{EMB_DIM}), dtype=float16")
print(f"[memmap] items:   shape=({i_count},{EMB_DIM}), dtype=float16")

[memmap] queries: shape=(690695,384), dtype=float16
[memmap] items:   shape=(5986464,384), dtype=float16


In [12]:
# ## 2) Утилиты: маппинг id→вектора и вычисление косинусов

def _lookup_indices(ids: List[int], id2idx: dict) -> np.ndarray:
    """
    Преобразует список числовых id в индексы меммапа.
    Если id не найден, кладём -1 — обработаем отдельно.
    """
    # JSON-ключи — строки
    out = np.fromiter((id2idx.get(str(x), -1) for x in ids), dtype=np.int64, count=len(ids))
    return out

def _fetch_vectors(mm: np.memmap, idxs: np.ndarray) -> np.ndarray:
    """
    Достаёт вектора из memmap по индексам. Для idx=-1 возвращает нулевой вектор.
    Возвращает float32 матрицу (для точности в dot).
    """
    n = len(idxs)
    # Инициализируем нулями (на случай -1)
    out = np.zeros((n, mm.shape[1]), dtype=np.float32)
    mask = idxs >= 0
    if mask.any():
        out[mask] = mm[idxs[mask]].astype(np.float32)  # float16 -> float32
    return out

def cosine_from_normalized(Q: np.ndarray, T: np.ndarray) -> np.ndarray:
    """
    Косинусы для L2-норм. векторов: cos = sum(Q*T, axis=1)
    """
    return (Q * T).sum(axis=1, dtype=np.float32)

def compute_chunk_cos(df: pl.DataFrame) -> pl.DataFrame:
    """
    Получает DataFrame с колонками [query_id, item_id] и возвращает тот же + cos_q_title.
    """
    qids = df.get_column("query_id").to_list()
    iids = df.get_column("item_id").to_list()

    qidx = _lookup_indices(qids, q_id2idx)
    iidx = _lookup_indices(iids, i_id2idx)

    Q = _fetch_vectors(q_mm, qidx)  # (n, d) float32
    T = _fetch_vectors(i_mm, iidx)  # (n, d) float32

    cos = cosine_from_normalized(Q, T)  # (n,)
    return df.with_columns(pl.Series("cos_q_title", cos))

In [13]:
# ## 3) Проход по train/test чанками и сохранение

def make_cos_parquet(in_path: str, out_path: str, chunk_rows: int = CHUNK_ROWS) -> str:
    """
    Сканирует parquet (только query_id, item_id), батчит, считает косинусы, складывает в один parquet.
    """
    lf = pl.scan_parquet(in_path).select(["query_id", "item_id"])
    n_rows = lf.select(pl.len()).collect().item()
    print(f"[run] {os.path.basename(in_path)} rows={n_rows}, chunk_rows={chunk_rows}")

    parts: list[pl.DataFrame] = []
    processed = 0

    with tqdm(total=n_rows, desc=f"cosine [{os.path.basename(in_path)}]") as pbar:
        for offset in range(0, n_rows, chunk_rows):
            length = min(chunk_rows, n_rows - offset)
            part = lf.slice(offset, length).collect()  # только 2 колонки
            part_out = compute_chunk_cos(part)         # добавили cos_q_title
            parts.append(part_out)
            processed += length
            pbar.update(length)

    out_df = pl.concat(parts, how="vertical")
    out_df.write_parquet(out_path, compression="zstd")
    print(f"[done] saved {out_path} shape={out_df.shape}")
    return out_path

In [14]:
# ## 4) Запуск

_ = make_cos_parquet(TRAIN_PATH, TRAIN_COS_OUT, chunk_rows=CHUNK_ROWS)
_ = make_cos_parquet(TEST_PATH, TEST_COS_OUT, chunk_rows=max(200_000, CHUNK_ROWS // 4))

[run] train-dset.parquet rows=7781790, chunk_rows=2000000


cosine [train-dset.parquet]: 100%|██████████| 7781790/7781790 [02:31<00:00, 51347.29it/s]


[done] saved train_cos.parquet shape=(7781790, 3)
[run] test-dset-small.parquet rows=335348, chunk_rows=500000


cosine [test-dset-small.parquet]: 100%|██████████| 335348/335348 [00:01<00:00, 251842.37it/s]

[done] saved test_cos.parquet shape=(335348, 3)


In [15]:
import polars as pl
import numpy as np

# 1) Загрузим исходные пары и результат
train_pairs = pl.scan_parquet(TRAIN_PATH).select(["query_id","item_id"]).collect(streaming=True)
test_pairs  = pl.scan_parquet(TEST_PATH ).select(["query_id","item_id"]).collect(streaming=True)

C:\Users\idine\AppData\Local\Temp\ipykernel_8476\3207270212.py:5: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  train_pairs = pl.scan_parquet(TRAIN_PATH).select(["query_id","item_id"]).collect(streaming=True)
C:\Users\idine\AppData\Local\Temp\ipykernel_8476\3207270212.py:6: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  test_pairs  = pl.scan_parquet(TEST_PATH ).select(["query_id","item_id"]).collect(streaming=True)


In [16]:
train_cos = pl.read_parquet(TRAIN_COS_OUT)
test_cos  = pl.read_parquet(TEST_COS_OUT)

print("[shape] rows (pairs) train orig vs cos:", train_pairs.height, train_cos.height)
print("[shape] rows (pairs) test  orig vs cos:", test_pairs.height,  test_cos.height)

[shape] rows (pairs) train orig vs cos: 7781790 7781790
[shape] rows (pairs) test  orig vs cos: 335348 335348


In [17]:
# 2) Проверим на дубликаты в результатах
dup_train = (train_cos.group_by(["query_id","item_id"])
             .agg(pl.len().alias("cnt")).filter(pl.col("cnt")>1))
dup_test  = (test_cos.group_by(["query_id","item_id"])
             .agg(pl.len().alias("cnt")).filter(pl.col("cnt")>1))

print("[dups] train duplicates:", dup_train.height)
print("[dups] test  duplicates:", dup_test.height)


[dups] train duplicates: 0
[dups] test  duplicates: 0


In [18]:
# 3) Анти-джойны: чего не хватило / лишнее
miss_train = train_pairs.join(train_cos, on=["query_id","item_id"], how="anti")
miss_test  = test_pairs.join(test_cos,  on=["query_id","item_id"], how="anti")

extra_train = train_cos.select(["query_id","item_id"]).join(train_pairs, on=["query_id","item_id"], how="anti")
extra_test  = test_cos.select(["query_id","item_id"]).join(test_pairs,  on=["query_id","item_id"],  how="anti")

print("[missing] train pairs without cos:", miss_train.height)
print("[missing] test  pairs without cos:", miss_test.height)
print("[extra]   train cos rows w/o source pair:", extra_train.height)
print("[extra]   test  cos rows w/o source pair:",  extra_test.height)

[missing] train pairs without cos: 0
[missing] test  pairs without cos: 0
[extra]   train cos rows w/o source pair: 0
[extra]   test  cos rows w/o source pair: 0


In [19]:
# 4) NaN / inf / диапазон косинуса
def cos_checks(df: pl.DataFrame, name: str):
    s = df.get_column("cos_q_title")
    nan_cnt = int(s.is_nan().sum())
    null_cnt = int(s.is_null().sum())
    inf_cnt = int((~s.is_finite()).sum())
    min_v = float(s.min())
    max_v = float(s.max())
    out_of_range = int(((s < -1.0001) | (s > 1.0001)).sum())
    print(f"[cos] {name}: NaN={nan_cnt}, NULL={null_cnt}, non-finite={inf_cnt}, "
          f"min={min_v:.4f}, max={max_v:.4f}, out_of_range={out_of_range}")

cos_checks(train_cos, "train")
cos_checks(test_cos,  "test")

[cos] train: NaN=0, NULL=0, non-finite=0, min=0.6754, max=0.9851, out_of_range=0
[cos] test: NaN=0, NULL=0, non-finite=0, min=0.6991, max=0.9792, out_of_range=0


In [20]:
# 5) Быстрый sanity-check распределения
print(train_cos.select([
    pl.col("cos_q_title").quantile(0.01).alias("p01"),
    pl.col("cos_q_title").quantile(0.50).alias("p50"),
    pl.col("cos_q_title").quantile(0.99).alias("p99"),
]).with_columns(pl.lit("train").alias("set")))
print(test_cos.select([
    pl.col("cos_q_title").quantile(0.01).alias("p01"),
    pl.col("cos_q_title").quantile(0.50).alias("p50"),
    pl.col("cos_q_title").quantile(0.99).alias("p99"),
]).with_columns(pl.lit("test").alias("set")))

shape: (1, 4)
┌─────────┬──────────┬──────────┬───────┐
│ p01     ┆ p50      ┆ p99      ┆ set   │
│ ---     ┆ ---      ┆ ---      ┆ ---   │
│ f32     ┆ f32      ┆ f32      ┆ str   │
╞═════════╪══════════╪══════════╪═══════╡
│ 0.76231 ┆ 0.871206 ┆ 0.949097 ┆ train │
└─────────┴──────────┴──────────┴───────┘
shape: (1, 4)
┌──────────┬──────────┬──────────┬──────┐
│ p01      ┆ p50      ┆ p99      ┆ set  │
│ ---      ┆ ---      ┆ ---      ┆ ---  │
│ f32      ┆ f32      ┆ f32      ┆ str  │
╞══════════╪══════════╪══════════╪══════╡
│ 0.766594 ┆ 0.870807 ┆ 0.948584 ┆ test │
└──────────┴──────────┴──────────┴──────┘


In [21]:
# 6) (опц.) Проверим, что все уникальные id покрыты
def coverage(orig: pl.DataFrame, cos: pl.DataFrame, name: str):
    uq_q_orig = int(orig.select(pl.col("query_id").n_unique()).item())
    uq_i_orig = int(orig.select(pl.col("item_id").n_unique()).item())
    uq_q_cos  = int(cos.select(pl.col("query_id").n_unique()).item())
    uq_i_cos  = int(cos.select(pl.col("item_id").n_unique()).item())
    print(f"[uniq] {name}: queries orig/cos = {uq_q_orig}/{uq_q_cos}, items orig/cos = {uq_i_orig}/{uq_i_cos}")

coverage(train_pairs, train_cos, "train")
coverage(test_pairs,  test_cos,  "test")

[uniq] train: queries orig/cos = 678190/678190, items orig/cos = 5773058/5773058
[uniq] test: queries orig/cos = 12505/12505, items orig/cos = 321625/321625


In [22]:
# 7) Покажем примеры, если что-то потерялось
if miss_train.height > 0:
    print("[sample missing train]:")
    print(miss_train.head(5))
if miss_test.height > 0:
    print("[sample missing test]:")
    print(miss_test.head(5))
if dup_train.height > 0:
    print("[sample duplicate train]:")
    print(dup_train.head(5))
if dup_test.height > 0:
    print("[sample duplicate test]:")
    print(dup_test.head(5))